In [ ]:
#| default_exp torch_core

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import math, torch

## CUDALongTensor

In [ ]:
#| export
class CUDALongTensor:
    """
    A wrapper class for `torch.cuda.LongTensor`. When performing operations that are
    currently not supported for `torch.cuda.LongTensor` (e.g `matmul`), it will
    convert the underlying LongTensor into DoubleTensor and convert the computed
    result back to a LongTensor. The computed result will be the same as the original
    expected result.
    """

    __BITS = torch.iinfo(torch.long).bits
    __DEFAULT_NBLOCKS = 3
    __BLOCK_SIZE = {3: None, 4: None}  # Number of bits per block
    __INDICES = {3: [], 4: []}
    __SHIFTS = {3: [], 4: []}
    for nblocks in [3, 4]:
        __BLOCK_SIZE[nblocks] = math.ceil(__BITS / nblocks)
        for i in range(nblocks):
            for j in range(nblocks):
                if (i + j) * __BLOCK_SIZE[nblocks] >= __BITS:
                    continue
                idx = i * nblocks + j
                __INDICES[nblocks].append(idx)
                __SHIFTS[nblocks].append((i + j) * __BLOCK_SIZE[nblocks])
                
    @staticmethod
    def __encode_as_fp64(x, num_blocks=3):
        """Converts a CUDALongTensor to an encoding of
        torch.cuda.DoubleTensor that represent the same data.
        """
        nb = num_blocks
        bks = CUDALongTensor.__BLOCK_SIZE[num_blocks]

        x_block = torch.stack(
            [(x >> (bks * i)) & (2**bks - 1) for i in range(nb)]
        )

        return x_block.double()

    @staticmethod
    def __decode_as_int64(x, num_blocks=3):
        """Converts a CUDALongTensor encoded as torch.cuda.DoubleTensor
        back to the CUDALongTensor it encodes
        """
        x = x.long()

        indices = CUDALongTensor.__INDICES[num_blocks]
        shifts = CUDALongTensor.__SHIFTS[num_blocks]

        indices = torch.tensor(indices, device=x.device)
        shifts = torch.tensor(shifts, device=x.device)
        shifts = shifts.view(-1, *([1] * (x.ndim - 1)))

        result = torch.index_select(x, 0, indices)
        result <<= shifts

        return result.sum(0)

    @staticmethod
    def matmul(x, y, *args, **kwargs):
        if not x.is_cuda or not y.is_cuda: return x@y
        
        # Use 4 blocks if each dot product is 256 elements or larger to prevent overflow in the sum
        nb = 3 if x.size(-1) < 256 else 4

        # Prepend 1 to the dimension of x or y if it is 1-dimensional
        remove_x, remove_y = False, False
        if x.dim() == 1:
            x = x.view(1, x.shape[0])
            remove_x = True
        if y.dim() == 1:
            y = y.view(y.shape[0], 1)
            remove_y = True

        x_encoded = CUDALongTensor.__encode_as_fp64(x, nb)
        y_encoded = CUDALongTensor.__encode_as_fp64(y, nb)

        # Span x and y for cross multiplication
        repeat_idx = [1] * (x_encoded.dim() - 1)
        x_enc_span = x_encoded.repeat(nb, *repeat_idx)
        y_enc_span = torch.repeat_interleave(y_encoded, repeats=nb, dim=0)

        # Broadcasting
        for _ in range(abs(x_enc_span.ndim - y_enc_span.ndim)):
            if x_enc_span.ndim > y_enc_span.ndim:
                y_enc_span.unsqueeze_(1)
            else:
                x_enc_span.unsqueeze_(1)

        z_encoded = torch.matmul(x_enc_span, y_enc_span, *args, **kwargs)

        if remove_x:
            z_encoded.squeeze_(-2)
        if remove_y:
            z_encoded.squeeze_(-1)

        return CUDALongTensor.__decode_as_int64(z_encoded, nb)


In [ ]:
u = torch.ones(4, 4, dtype=torch.int64).triu()
v = torch.randint(1, 10, size=(1,4))

In [ ]:
CUDALongTensor.matmul(v, u)

tensor([[ 2,  5, 14, 15]])

In [ ]:
u,v = u.to('cuda'), v.to('cuda')
CUDALongTensor.matmul(v, u)

tensor([[ 2,  5, 14, 15]], device='cuda:0')